# Visualize and print results on test set 

In [ ]:
from pathlib import Path
import pandas as pd
import json
from collections import defaultdict

pd.options.plotting.backend = "plotly"


### CHANGE THIS ###
results_dirname = "../output/evaluering/"
### ---------- ###

p = Path(results_dirname)

plot_data = defaultdict(list)
score_data = defaultdict(list)

for model_dir in p.iterdir():
    model_name = model_dir.name
    collection_level = json.loads((model_dir / "all_pages.json").read_text())

    score_data["model"].append(model_name)

    for score_name, value in collection_level.items():
        plot_data["model"].append(model_name)
        plot_data["score"].append(score_name)
        plot_data["value"].append(value)

        # add inverted score
        plot_data["model"].append(model_name)
        plot_data["score"].append(f"1 - {score_name}")
        plot_data["value"].append(1 - value)

        score_data[score_name].append(value)


plot_data_df = pd.DataFrame(plot_data)
plot_df = plot_data_df[plot_data_df.score.apply(lambda x: "1" in x)]
plot_df.plot.bar(x="score", y="value", color="model", barmode="group")

In [ ]:
df = pd.DataFrame(score_data)
df.sort_values("CER")